In [1]:
import data.ASSISTments2009.assist09_wrangler as assist09_wrangler
import data.ASSISTments2012.assist12_wrangler as assist12_wrangler
import data.junyi2015.junyi15_wrangler as junyi15_wrangler
import data.NeurIPS2020.neur20_wrangler as neur20_wrangler
import data.dataSplitter as ds
import time
import numpy as np
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import os

In [2]:
from models import IRT, NCDM, KaNCD, CAKE

paths = {
    "ASSIST09" : "data/ASSISTments2009/",
    "NEUR20" : "data/NeurIPS2020/",
    "JUNYI15" : "data/junyi2015/",
    "ASSIST12": "data/ASSISTments2012/"
}

modelList ={
    "0":["IRT", "NCDM", "KaNCD", "CAKE"],
    "1":["IRT"],
    "2":["NCDM"],
    "3":["KaNCD"],
    "4":["CAKE"]
}

datasetList = {
    "0":["ASSIST09", "NEUR20", "JUNYI15", "ASSIST12"],
    "1":["ASSIST09"],
    "2":["NEUR20"],
    "3":["JUNYI15"],
    "4":["ASSIST12"]
}

runTypeList = {
    "0":["basic", "sampled", "undersampleEven", "undersampleCorrect", "undersampleIncorrect"],
    "1":["basic"],
    "2":["sampled"],
    "3":['undersampleEven'],
    "4":["undersampleCorrect"],
    "5":["undersampleIncorrect"],
    "6":["correctSaturated"],
    "7":["incorrectSaturated"],
    
}

modelFuncs = {
    "IRT": IRT.run_IRT,
    "NCDM":NCDM.run_NCDM,
    "KaNCD":KaNCD.run_KaNCD,
    "CAKE":CAKE.run_CAKE
}
models = modelList[input(f"Select a model to test:\n" + "\n".join([f'{k}.{modelList[k] if len(modelList[k]) == 1 else "all"}' for k in modelList.keys()]) + "\n")]
datasets = datasetList[input(f"Select a dataset to test on:\n" + "\n".join([f'{k}.{datasetList[k] if len(datasetList[k]) == 1 else "all"}' for k in datasetList.keys()]) + "\n")]
runTypes = runTypeList[input(f"Select a run condition:\n" + "\n".join([f'{k}.{runTypeList[k] if len(runTypeList[k]) == 1 else "all"}' for k in runTypeList.keys()]) + "\n")]
user_data = True if input("Collect user data?\n") == 'y' else False

Select a model to test:
0.all
1.['IRT']
2.['NCDM']
3.['KaNCD']
4.['CAKE']
 3
Select a dataset to test on:
0.all
1.['ASSIST09']
2.['NEUR20']
3.['JUNYI15']
4.['ASSIST12']
 1
Select a run condition:
0.all
1.['basic']
2.['sampled']
3.['undersampleEven']
4.['undersampleCorrect']
5.['undersampleIncorrect']
6.['correctSaturated']
7.['incorrectSaturated']
 1
Collect user data?
 n


In [3]:
results_path = "results.csv"

for model in models:
    for dataset in datasets:
        for runType in runTypes:
            #if the csv doesn't exist create it
            if not os.path.exists(results_path):
                df = pd.DataFrame(columns=[
                    "model", "runType", "dataset",
                    "test_correct_ratio", "test_correct_ratio_raw",
                    "train_correct_ratio", "train_correct_ratio_raw",
                    "n_train_examples", "n_train_examples_raw",
                    "n_test_examples", "n_test_examples_raw",
                    "n_valid_examples", "n_valid_examples_raw",
                    "n_knowledge_concepts", "n_problems",
                    "n_users", "n_users_raw",
                    "ACC", "ACC_raw", "AUC", "AUC_raw",
                    "MAE", "MAE_raw", "RMSE", "RMSE_raw",
                    "ACC_std", "ACC_std_raw", "AUC_std", "AUC_std_raw",
                    "MAE_std", "MAE_std_raw", "RMSE_std", "RMSE_std_raw",
                    "avg_train_duration", "avg_train_duration_raw",
                    "performed_at"
                ])
            else:
                #pull in the csv as a dataframe
                df = pd.read_csv(results_path)
            #check the presence of an existing run
            if (((df['model'] == model) & (df['runType'] == runType) & (df['dataset'] == dataset)).any()) and not user_data:
                #data entry already exists, skip
                print("data exists in result file")
            else:
                Q, data = ds.prepareData(dataset = dataset, runType = runType)
                #find average training correct/incorrect ratio per student (should be ~0.50)
                avg_train_score = []
                avg_test_score = []
                avg_valid_score = []
                avg_train_size = []
                avg_test_size = []
                avg_valid_size = []
                avg_n_users = []
                for d in data:
                    avg_train_score.append(d["train"].groupby('user_id')["score"].mean().mean())
                    avg_test_score.append(d["test"].groupby('user_id')["score"].mean().mean())
                    avg_valid_score.append(d["valid"].groupby('user_id')["score"].mean().mean())
                    avg_train_size.append(len(d["train"]))
                    avg_test_size.append(len(d["test"]))
                    avg_valid_size.append(len(d["valid"]))
                    avg_n_users.append(len(d["train"]["user_id"].unique()))
                
                result_object = {
                    "model" : model,
                    "runType" : runType,
                    "dataset" : dataset,
                    "test_correct_ratio":np.mean(avg_test_score),
                    "test_correct_ratio_raw":avg_test_score,
                    "train_correct_ratio":np.mean(avg_train_score),
                    "train_correct_ratio_raw":avg_train_score,
                    "n_train_examples":np.mean(avg_train_size),
                    "n_train_examples_raw":avg_train_size,
                    "n_test_examples":np.mean(avg_test_size),
                    "n_test_examples_raw":avg_test_size,
                    "n_valid_examples":np.mean(avg_valid_size),
                    "n_valid_examples_raw":avg_valid_size,
                    "n_knowledge_concepts":len(Q.columns) - 1,
                    "n_problems":len(Q),
                    "n_users":np.mean(avg_n_users),
                    "n_users_raw":avg_n_users,
                }
                
                accs, aucs, maes, rmses, times = [], [], [], [], []
                
                #run each provided data configuration and collect statistics
                
                for n, run in enumerate(data): 
                    
                    start_timer = time.time()
                    acc, auc, mae, rmse = modelFuncs[model](Q, run["train"], run["test"], run["valid"], n, dataset, runType, user_data = user_data)
                    end_timer = time.time()
                    
                    accs.append(acc)
                    aucs.append(auc)
                    maes.append(mae)
                    rmses.append(rmse)
                    times.append(end_timer - start_timer)

                    print(f"\nRan fold {n} on {model} with {dataset} and run type {runType}\n")

                #collecting user data will slow the train time
                if not user_data:
                    #format the output
                    result_object['ACC'] = np.mean(accs)
                    result_object['ACC_raw'] = accs
                    result_object['AUC'] = np.mean(aucs)
                    result_object['AUC_raw'] = aucs
                    result_object['MAE'] = np.mean(maes)
                    result_object['MAE_raw'] = maes
                    result_object['RMSE'] = np.mean(rmses)
                    result_object['RMSE_raw'] = rmses
                    result_object['ACC_std'] = np.std(accs)
                    result_object['ACC_std_raw'] = accs
                    result_object['AUC_std'] = np.std(aucs)
                    result_object['AUC_std_raw'] = aucs
                    result_object['MAE_std'] = np.std(maes)
                    result_object['MAE_std_raw'] = maes
                    result_object['RMSE_std'] = np.std(rmses)
                    result_object['RMSE_std_raw'] = rmses
                    result_object['avg_train_duration'] = np.mean(times)
                    result_object['avg_train_duration_raw'] = times
                    result_object['performed_at'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
                    df = pd.concat([df, pd.DataFrame([result_object])], ignore_index=True)
                    
                    df.to_csv(results_path, index=False)
                
                print(f"\nVVVVVVVVVV\nRan {model} on {dataset} type {runType}\n^^^^^^^^^^\n")

data exists in result file
